In [1]:
from torch.backends import cudnn
import time


class Client:
    def __init__(self, client_id, data_loader, model, device):
        """
        Initializes a federated learning client.
        :param client_id: Unique identifier for the client.
        :param data_loader: Data loader specific to the client.
        :param model: The model class to be used by the client.
        :param device: The device (CPU/GPU) to perform computations.
        """
        self.client_id = client_id
        self.data_loader = data_loader
        self.model = model.to(device)
        self.device = device

    def client_update(self, client_data, criterion, optimizer, local_steps=4, detailed_print=False):
        """
        Trains a given client's local model on its dataset for a fixed number of steps (`local_steps`).

        Args:
            model (nn.Module): The local model to be updated.
            client_id (int): Identifier for the client (used for logging/debugging purposes).
            client_data (DataLoader): The data loader for the client's dataset.
            criterion (Loss): The loss function used for training (e.g., CrossEntropyLoss).
            optimizer (Optimizer): The optimizer used for updating model parameters (e.g., SGD).
            local_steps (int): Number of local epochs to train on the client's dataset.
            detailed_print (bool): If True, logs the final loss after training.

        Returns:
            dict: The state dictionary of the updated model.
        """


        cudnn.benchmark  # Calling this optimizes runtime

        self.model.train()  # Set the model to training mode
        step_count = 0
        total_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        while step_count < local_steps:
            for data, targets in client_data:

                # Move data and targets to the specified device (e.g., GPU or CPU)
                data, targets = data.to(self.device), targets.to(self.device)

                # Reset the gradients before backpropagation
                optimizer.zero_grad()

                # Forward pass: compute model predictions
                outputs = self.model(data)

                # Compute the loss
                loss = criterion(outputs, targets)

                # Backward pass: compute gradients and update weights
                loss.backward()
                optimizer.step()

                #---------- Accumulate metrics
                #  Accumulates the weighted loss for the number of samples in the batch to account for any variation in
                #  batch size due to, for example, the smaller final batch. A little too precise? :)
                total_loss += loss.item() * data.size(0)
                _, predicted = outputs.max(1)
                correct_predictions += predicted.eq(targets).sum().item()
                total_samples += data.size(0)

                step_count +=1
                if step_count >= local_steps:
                  break

        #---------- Compute averaged metrics
        avg_loss = total_loss / total_samples
        avg_accuracy = correct_predictions / total_samples * 100

        # Optionally, print the loss for the last epoch of training
        if detailed_print:
          print(f'Client {self.client_id} --> Final Loss (Step {step_count}/{local_steps}): {loss.item()}')


        # Return the updated model's state dictionary (weights)
        return self.model.state_dict(), avg_loss, avg_accuracy

In [16]:
import torch
import torch.optim as optim
from copy import deepcopy
import numpy as np
from torch.utils.data import Subset
import os
from torch.utils.data import DataLoader, Subset
#from utils.utils import evaluate
#from utils.checkpointing_utils import save_checkpoint, load_checkpoint
import logging

log = logging.getLogger(__name__)

class Server:
    def __init__(self, global_model, device, CHECKPOINT_DIR):
        self.global_model = global_model
        self.device = device
        self.CHECKPOINT_DIR = CHECKPOINT_DIR
        # Ensure the checkpoint directory exists, creating it if necessary
        os.makedirs(CHECKPOINT_DIR, exist_ok=True)

    def fedavg_aggregate(self, client_states, client_sizes, client_avg_losses, client_avg_accuracies):
        """
        Aggregates model updates and client metrics from selected clients using the Federated Averaging (FedAvg) algorithm.
        The updates and metrics are weighted by the size of each client's dataset.

        Args:
            global_model (nn.Module): The global model whose structure is used for aggregation.
            client_states (list[dict]): A list of state dictionaries (model weights) from participating clients.
            client_sizes (list[int]): A list of dataset sizes for the participating clients.
            client_avg_losses (list[float]): A list of average losses for the participating clients.
            client_avg_accuracies (list[float]): A list of average accuracies for the participating clients.

        Returns:
            tuple: The aggregated state dictionary with updated model parameters, global average loss, and global average accuracy.
        """
        # Copy the global model's state dictionary for aggregation
        new_state = deepcopy(self.global_model.state_dict())

        # Calculate the total number of samples across all participating clients
        total_samples = sum(client_sizes)

        # Initialize all parameters in the new state to zero
        for key in new_state:
            new_state[key] = torch.zeros_like(new_state[key])

        # Initialize metrics
        total_loss = 0.0
        total_accuracy = 0.0

        # Perform a weighted average of client updates and metrics
        for state, size, avg_loss, avg_accuracy in zip(client_states, client_sizes, client_avg_losses, client_avg_accuracies):
            for key in new_state:
                # Add the weighted contribution of each client's parameters
                new_state[key] += (state[key] * size / total_samples)
            total_loss += avg_loss * size
            total_accuracy += avg_accuracy * size

        # Calculate global metrics
        global_avg_loss = total_loss / total_samples
        global_avg_accuracy = total_accuracy / total_samples

        # Return the aggregated state dictionary with updated weights and global metrics
        return new_state, global_avg_loss, global_avg_accuracy


    # Federated Learning Training Loop
    ### UPDATED
    #0. receives the shards (the result of sharding method)
    #1. receive the n clients to perform one round
    #2. averages the results coming from the model of the n clients
    #3. returns the model and the results averaged.
    def train_federated(self, criterion, lr, momentum, batchsize, wd, selected_clients, shards, local_steps=4):
      # selected_clients are objects of Individual class

        train_accuracies = []
        train_losses = []

        #shards = self.sharding(trainloader.dataset, num_clients, num_classes) #each shard represent the training data for one client
        client_sizes = [len(shard) for shard in shards]

        self.global_model.to(self.device) #as alwayse, we move the global model to the specified device (CPU or GPU)

        # we only mantain a round
        client_states = []
        client_avg_losses = []
        client_avg_accuracies = []

        # 2) local training: for each client updates the model using the client's data for local_steps epochs
        for client_id in selected_clients.genome:
            local_model = deepcopy(self.global_model) #it creates a local copy of the global model
            optimizer = optim.SGD(local_model.parameters(), lr=lr, momentum=momentum, weight_decay=wd) #same of the centralized version
            client_loader = DataLoader(shards[client_id], batch_size=batchsize, shuffle=True)

            client = Client(client_id, client_loader, local_model, self.device)
            client_local_state, client_avg_loss, client_avg_accuracy  = client.client_update(client_loader, criterion, optimizer, local_steps)

            client_states.append(client_local_state)
            client_avg_losses.append(client_avg_loss)
            client_avg_accuracies.append(client_avg_accuracy)


        # 3) central aggregation: aggregates participating client updates using fedavg_aggregate
        #    and replaces the current parameters of global_model with the returned ones.
        aggregated_state, train_loss, train_accuracy = self.fedavg_aggregate(client_states, [client_sizes[i] for i in selected_clients.genome], client_avg_losses, client_avg_accuracies)
        # UPDATE THE FITNESS OF INDIVIDUAL:
        selected_clients.fitness = train_accuracy

        self.global_model.load_state_dict(aggregated_state)

        train_accuracies.append(train_accuracy)
        train_losses.append(train_loss)


        return self.global_model.state_dict(), train_accuracy, train_loss

    def skewed_probabilities(self, number_of_clients, gamma=0.5):
            # Generate skewed probabilities using a Dirichlet distribution
            probabilities = np.random.dirichlet(np.ones(number_of_clients) * gamma)
            return probabilities

    def client_selection(self,number_of_clients, clients_fraction, probabilities=None):
        """
        Selects a subset of clients based on uniform or skewed distribution.

        Args:
        number_of_clients (int): Total number of clients.
        clients_fraction (float): Fraction of clients to be selected.
        uniform (bool): If True, selects clients uniformly. If False, selects clients based on a skewed distribution.
        gamma (float): Hyperparameter for the Dirichlet distribution controlling the skewness (only used if uniform=False).

        Returns:
        list: List of selected client indices.
        """
        num_clients_to_select = int(number_of_clients * clients_fraction)

        if probabilities is None:
            # Uniformly select clients without replacement
            selected_clients = np.random.choice(number_of_clients, num_clients_to_select, replace=False)
        else:
            selected_clients = np.random.choice(number_of_clients, num_clients_to_select, replace=False, p=probabilities)

        return selected_clients



    def sharding(self, dataset, number_of_clients, number_of_classes=100):
        """
        Function that performs the sharding of the dataset given as input.
        dataset: dataset to be split (should be a PyTorch dataset or similar);
        number_of_clients: the number of partitions we want to obtain (e.g., 100 for 100 clients);
        number_of_classes: (int) the number of classes inside each partition, or 100 for IID (default to 100).
        """

        # Validate the number of classes input
        if not (1 <= number_of_classes <= 100):
            raise ValueError("number_of_classes should be an integer between 1 and 100")

        # Shuffle dataset indices for randomness
        indices = np.random.permutation(len(dataset))

        if number_of_classes == 100:  # IID Case
            # Equally distribute indices among clients: we can just randomly assign an equal number of records to each client

            # Compute basic partition sizes
            basic_partition_size = len(dataset) // number_of_clients
            remainder = len(dataset) % number_of_clients

            shards = []  # This will hold the final dataset shards
            start_idx = 0

            for i in range(number_of_clients):
                end_idx = start_idx + basic_partition_size + (1 if i < remainder else 0)
                shards.append(Subset(dataset, indices[start_idx:end_idx]))
                start_idx = end_idx
            return shards

        else:  # non-IID Case
            # Get labels for sorting
            labels = np.array([dataset[i][1] for i in range(len(dataset))])
            TOTAL_NUM_CLASSES = len(set(labels))

            shard_size = len(dataset) // (number_of_clients * number_of_classes)  # Shard size for each class per client
            print("dataset len: ", len(dataset), ", shard size: ", shard_size, ", number of shards: ",(number_of_clients * number_of_classes))
            if shard_size == 0:
                raise ValueError("Shard size is too small; increase dataset size or reduce number of clients/classes.")


            # Divide the dataset into shards, each containing samples from one class
            shards = {}
            for i in range(TOTAL_NUM_CLASSES):
                # Filter samples for the current class
                class_samples = [j for j in range(len(labels)) if labels[j] == i]
                shards_of_class_i = []
                # While there are enough samples to form a shard
                while len(class_samples) >= shard_size:
                    # Take a shard of shard_size samples
                    shards_of_class_i.append(class_samples[:shard_size])
                    # Remove the shard_size samples from class_samples
                    class_samples = class_samples[shard_size:]
                # Add the last shard (which might be smaller than shard_size)
                if class_samples:
                    shards_of_class_i.append(class_samples)
                # Store the class shards
                shards[i] = shards_of_class_i  # Store shards by class

            client_shards = []  # List to store the dataset for each client
            for client_id in range(number_of_clients):

                client_labels = [label % TOTAL_NUM_CLASSES for label in range(client_id, client_id + number_of_classes)]
                #print(client_labels)

                # Collect the shards for the selected classes
                client_shard_indices = []
                for label in client_labels:
                    shard = shards[label].pop(0)  # Pop the first shard from the class's shard list
                    client_shard_indices.append(shard)

                # Flatten and combine the shard indices into one list
                client_indices = [idx for shard in client_shard_indices for idx in shard]

                #print(f"Client {client_id} has {len(client_indices)} samples divided in {len(client_shard_indices)} shards (classes).")
                # Create a Subset for the client
                client_dataset = Subset(dataset, client_indices)
                client_shards.append(client_dataset)

            return client_shards  # Return the list of dataset subsets (shards) for each client



In [3]:
import torch.nn as nn
import torch.nn.functional as F
"""
Model architecture for the CIFAR-100 dataset.
The model is based on the LeNet-5 architecture with some modifications.
Reference: Hsu et al., Federated Visual Classification with Real-World Data Distribution, ECCV 2020

CNN similar to LeNet5 which has two 5×5, 64-channel convolution layers, each precedes a 2×2
max-pooling layer, followed by two fully-connected layers with 384 and 192
channels respectively and finally a softmax linear classifier
"""


class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 5 * 5, 384),  # Updated to be consistent with data augmentation
            nn.ReLU(),
            nn.Linear(384, 192),
            nn.ReLU(),
            nn.Linear(192, 100)  # 100 classes for CIFAR-100
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)  # Flatten the output of the conv layers
        x = self.fc_layer(x)
        x = F.log_softmax(x, dim=1)
        return x


In [4]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

class CIFAR100DataLoader:
    def __init__(self, batch_size=64, validation_split=0.1, download=True, num_workers=0, pin_memory=True):
        self.batch_size = batch_size
        self.validation_split = validation_split
        self.download = download
        self.num_workers = num_workers
        self.pin_memory = pin_memory

        # Define transformations
        self.train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
        ])

        self.test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
        ])

        # Load datasets
        self.train_loader, self.val_loader, self.test_loader = self._prepare_loaders()

    def _prepare_loaders(self):
        # Load the full training dataset
        full_trainset = datasets.CIFAR100(root='./data', train=True, download=self.download, transform=self.train_transform)

        # Split indices for training and validation
        indexes = list(range(len(full_trainset)))
        train_indexes, val_indexes = train_test_split(
            indexes,
            train_size=1 - self.validation_split,
            test_size=self.validation_split,
            random_state=42,
            stratify=full_trainset.targets,
            shuffle=True
        )

        # Create training and validation subsets
        train_dataset = Subset(full_trainset, train_indexes)
        train_loader = DataLoader(
            train_dataset, batch_size=self.batch_size, shuffle=True,
            num_workers=self.num_workers, pin_memory=self.pin_memory
        )

        full_trainset_val = datasets.CIFAR100(root='./data', train=True, download=self.download, transform=self.test_transform)
        val_dataset = Subset(full_trainset_val, val_indexes)
        val_loader = DataLoader(
            val_dataset, batch_size=self.batch_size, shuffle=False,
            num_workers=self.num_workers, pin_memory=self.pin_memory
        )

        # Load the test dataset
        testset = datasets.CIFAR100(root='./data', train=False, download=self.download, transform=self.test_transform)
        test_loader = DataLoader(
            testset, batch_size=self.batch_size, shuffle=False,
            num_workers=self.num_workers, pin_memory=self.pin_memory
        )

        return train_loader, val_loader, test_loader

    def __iter__(self):
        """Allows iteration over all loaders for unified access."""
        return iter([self.train_loader, self.val_loader, self.test_loader])

In [5]:
import random


class Individual:
    def __init__(self, genome, total_clients=100,number_selected_clients = 2):
        """
        Initialize an Individual.

        :param genome: List of selected clients (subset of integers).
        :param total_clients: Total number of available clients (default 100).
        :param number_selected_clients: Number of clients to be selected (default 10).
        """
        self.genome = genome
        self.fitness = None  # Fitness will be computed separately
        self.total_clients = total_clients
        self.number_selected_clients = number_selected_clients

    def set_fitness(self, fitness_value):
        """
        Set the fitness value for the individual.

        :param fitness_value: Float value representing the fitness (e.g., loss or accuracy).
        """
        self.fitness = fitness_value

    def point_mutation(self):
        """
        Mutate the genome by changing 1 client randomly.
        Ensures that the selected clients remain disjoint.
        """
        num_changes = 1  # Number of mutations
        available_clients = set(range(self.total_clients)) - set(self.genome)  # Clients not in genome

        # Remove random clients from the genome
        to_remove = random.sample(self.genome, k=num_changes)
        for client in to_remove:
            self.genome.remove(client)

        # Add new random clients from the available set
        to_add = random.sample(list(available_clients), k=num_changes)
        self.genome.extend(to_add)

    @staticmethod
    def crossover(parent1, parent2):
        """
        Perform crossover between two parents.
        Select half genes from parent1 and half from parent2.
        Ensures that the genome is disjoint and valid.

        :param parent1: First parent Individual.
        :param parent2: Second parent Individual.
        :return: New Individual (offspring).
        """
        half_size = len(parent1.genome) // 2

        # Randomly select half genes from each parent
        genome1_part = random.sample(parent1.genome, k=half_size)
        genome2_part = [gene for gene in parent2.genome if gene not in genome1_part]

        # Combine to form new genome
        new_genome = genome1_part + genome2_part[:len(parent1.genome) - half_size]

        #Ensure that the genome is long enough
        if(len(new_genome)< parent1.number_selected_clients):
            new_genome = new_genome + random.sample(range(parent1.total_clients), k=parent1.number_selected_clients - len(new_genome))

        return Individual(genome=new_genome, total_clients=parent1.total_clients)

In [19]:
import random
from copy import deepcopy
import os
import torch
import torch.nn as nn
from statistics import mean
import numpy as np


def tournament_selection(population, tau=2):
    """
    Perform tournament selection to choose parents.
    Randomly select tau individuals and choose the best one.

    :param population: List of Individuals.
    :param tau: Number of individuals to select.
    :return: Selected Individual.
    """
    participants = random.sample(population, tau)
    winner = max(participants, key=lambda ind: ind.fitness)
    return deepcopy(winner)

def client_size(individual, client_sizes):
    """
    Computes the number of total samples for individual
    """
    val = 0
    for client in individual.genome:
        val += client_sizes[client]
    return val

def evaluate(model, dataloader, DEVICE, criterion):
    with torch.no_grad():
        model.train(False) # Set Network to evaluation mode
        running_corrects = 0
        losses = []
        for data, targets in dataloader:
            data = data.to(DEVICE)        # Move the data to the GPU
            targets = targets.to(DEVICE)  # Move the targets to the GPU
            # Forward Pass
            outputs = model(data)
            loss = criterion(outputs, targets)
            losses.append(loss.item())
            # Get predictions
            _, preds = torch.max(outputs.data, 1)
            # Update Corrects
            running_corrects += torch.sum(preds == targets.data).data.item()
            # Calculate Accuracy
            accuracy = running_corrects / float(len(dataloader.dataset))

    return accuracy*100, mean(losses)


def EA_algorithm(generations,population_size,num_clients,crossover_probability, dataset, valid_loader):
    """
    Perform the Evolutionary Algorithm (EA) to optimize the selection of clients.
    The EA consists of the following steps:
    1. Initialization: Create a population of individuals.
    2. Evaluation: Compute the fitness of each individual.
    3. Selection: Choose parents based on their fitness.
    4. Offspring to create the new population (generational model).
    6. Repeat from step 2 maximum iterations.

    :param generations: Number of generations to run the algorithm.
    :param population_size: Number of individuals in the population.
    :param num_clients: clients selected by each individual.
    :param crossover_probability: Probability of crossover for each individual.

    :return global_model: The global model obtained after the EA.
    :return global_accuracy: The validation accuracy of the global model at each generation.
    :return global_loss: The validation loss of the global model at each generation.
    return training_loss: The training loss of the global model at each generation.
    return training_accuracy: The training accuracy of the global model at each generation.
    """

    # Initialize the population
    population = [Individual(genome=random.sample(range(100), k=num_clients)) for _ in range(population_size)]
    model = LeNet5()
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    criterion = nn.NLLLoss()
    # Create the Server instance:
    server = Server(model,DEVICE,"server_data")
    num_classes = 100
    shards = server.sharding(dataset.dataset, num_clients, num_classes)
    client_sizes = [len(shard) for shard in shards]
    LR = 0.01
    batchsize = 64
    WD = 0.001
    MOMENTUM = 0.9


    for i in range(generations):
        # Select randomly 3 individuals:
        selected_individuals = population
        # For each of them apply the fed_avg algorithm:

        param_list = []
        averages_acc = []
        average_loss = []
        for choosen_individual in selected_individuals:
            resulting_model, acc_res, loss_res = server.train_federated(criterion, LR, MOMENTUM, batchsize, WD, choosen_individual, shards)
            param_list.append(resulting_model)
            averages_acc.append(acc_res)
            average_loss.append(loss_res)


        #Here we should average all the models to obtain the global model...
        averaged_model,  global_avg_loss, global_avg_accuracy = server.fedavg_aggregate(param_list, [client_size(i, client_sizes) for i in selected_individuals], average_loss, averages_acc)
        # Update the model with the result of the average:
        model.load_state_dict(averaged_model)


        # Then evaluate the validation accuracy of the global model
        acc, loss = evaluate(model, valid_loader, DEVICE, criterion)

        print(f"Generation {i+1}, accuracy {acc}, loss {loss}")

        offspring = []
        #Offspring-> offspring size is the same as population size
        for i in range(population_size):
            # Crossover
            if random.random() < crossover_probability:
                parent1 = tournament_selection(population)
                parent2 = tournament_selection(population)
                offspring.append(Individual.crossover(parent1, parent2))
            else:
                parent = tournament_selection(population)
                #parent.point_mutation()
                offspring.append(parent)

        # Replace the population with the new offspring
        population = offspring



if __name__ == '__main__':
    #10% of the dataset kept for validation
    BATCH_SIZE = 64
    data_loader = CIFAR100DataLoader(batch_size=BATCH_SIZE, validation_split=0.1, download=True, num_workers=4, pin_memory=True)
    trainloader, validloader, testloader = data_loader.train_loader, data_loader.val_loader, data_loader.test_loader
    EA_algorithm(10, 3, 100, 0.7, trainloader, validloader)



Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
Generation 1, accuracy 1.8800000000000001, loss 4.605861941470375


ValueError: Sample larger than population or is negative